In [1]:
import numpy as np

In [60]:
class Board:
    board = []    # Tablero
    n = None      # Cantidad de reinas
    parent = None # Apuntador al nodo padre
    queens = None   # Posicion de las reinas
    constrataints = None #Posiciones que no pueden usarse
    
    def __init__(self,n,board,parent = None,queens = [], constrataints = []):
        
        self.n = n
        self.board = board
        self.parent = parent
        self.queens = queens
        self.constrataints = constrataints
    
    def getConstrataints(self,pos):
        # Esta funcion obtiene las posiciones que no pueden ser usadas
        (i,j) = pos
        print "Hello",pos   
        
        #Agrega todos los de la misma columna
        while i < self.n:
            if not (i,j) in self.constrataints:
                self.constrataints.append((i,j))
            i+=1
        
        (i,j) = pos
        while i >= 0:
            if not (i,j) in self.constrataints:
                self.constrataints.append((i,j))
            i-=1
        
        #Agrega las posiciones de la misma fila
        (i,j) = pos
        while j < self.n:
            if not (i,j) in self.constrataints:
                self.constrataints.append((i,j))
            j+=1
        
        (i,j) = pos
        while j >= 0:
            if not (i,j) in self.constrataints:
                self.constrataints.append((i,j))
            j-=1
            
        #Agrega las posiciones de la diagonal izquierda hacia arriba
        (i,j) = pos
        while j >= 0 and i >= 0:
            if not (i,j) in self.constrataints:
                self.constrataints.append((i,j))
            j-=1
            i-=1
        
        #Agrega las posiciones de la diagonal izquierda hacia arriba
        (i,j) = pos
        while j >= 0 and i < n:
            if not (i,j) in self.constrataints:
                self.constrataints.append((i,j))
            j-=1
            i+=1
            
        (i,j) = pos
        while j < n and i >= 0:
            if not (i,j) in self.constrataints:
                self.constrataints.append((i,j))
            j+=1
            i-=1
            
        #Agrega las posiciones de la diagonal derecha hacia abajo
        (i,j) = pos
        while j < n and i <n:
            if not (i,j) in self.constrataints:
                self.constrataints.append((i,j))
            j+=1
            i+=1
        self.constrataints = sorted(self.constrataints,key= lambda x: x[0])
        print self.constrataints
    
    
    
    def getNeighbours(self):
        print self.board       
        

        
n = 4
n_queens = Board(n,np.zeros((n,n)))
n_queens.getConstrataints((0,0))

Hello (0, 0)
[(0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (2, 0), (2, 2), (3, 0), (3, 3)]


In [57]:
n_queens.board[0,0] = 1
n_queens.board

array([[1., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])